In [233]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import folium
import colou

from sklearn.preprocessing import normalize
from sklearn.base import ClusterMixin
from sklearn.cluster import KMeans, SpectralClustering, DBSCAN

import pickle

np.random.seed(777)

In [196]:
data = pd.read_excel('data/City surface public transport stops.xlsx')
data = data[data.AdmArea_en == "Czentral`ny'j administrativny'j okrug"]
data = data.reset_index()
data.head()

,index,ID_en,Name_en,Longitude_WGS84_en,Latitude_WGS84_en,Street_en,AdmArea_en,District_en,RouteNumbers_en,StationName_en,Direction_en,Pavilion_en,OperatingOrgName_en,EntryState_en,global_id,geoData
0,278,347,"«2-й Лесной пер.», улица Бутырский Вал (347)",37.586584,55.782106,улица Бутырский Вал,Czentral`ny'j administrativny'j okrug,Tverskoj rajon,АТ18; А12; АТ56; АТ78,2-й Лесной пер.,NaN,да,ГУП «Мосгортранс»,active,889085436,NaN
1,279,349,"«Ул. Сущевский Вал», Новослободская улица (349)",37.590714,55.790630,Новослободская улица,Czentral`ny'j administrativny'j okrug,Tverskoj rajon,АТ47; АМ10; АТ3; АТ56; АТ78,Ул. Сущевский Вал,NaN,да,ГУП «Мосгортранс»,active,889085437,NaN
2,355,479,"«Стадион Лужники (южн.) (пос.)», улица Лужники...",37.565972,55.714265,улица Лужники,Czentral`ny'j administrativny'j okrug,rajon Xamovniki,А64; А216; А809; АС12; А255; АБК; АТ79,Стадион Лужники (южн.) (пос.),NaN,да,ГУП «Мосгортранс»,active,889085549,NaN
3,356,480,"«Спортзал Дружба», Лужнецкая набережная (480)",37.570191,55.712504,Лужнецкая набережная,Czentral`ny'j administrativny'j okrug,rajon Xamovniki,А64; А216; А809; АС12; А255; АБК; АТ79,Спортзал Дружба,NaN,да,ГУП «Мосгортранс»,active,889085550,NaN
4,357,481,"«Лужнецкая наб.», Лужнецкая набережная (481)",37.574559,55.713770,Лужнецкая набережная,Czentral`ny'j administrativny'j okrug,rajon Xamovniki,А64; А216; А809; АС12; А255; АБК; АТ79,Лужнецкая наб.,NaN,да,ГУП «Мосгортранс»,active,889085551,NaN


map = folium.Map([55.75215, 37.61819], zoom_start=12)
for id, row in data.iterrows():
    folium.Circle([row.Latitude_WGS84_en, row.Longitude_WGS84_en],
                  radius=10).add_to(map)
map

In [197]:
def get_routes(data):
    '''
    Accumulate routes from raw data
    param data: pd.DataFrame - public transport stops data
    return: dict - unsorted stops ids for each route,
                   e.g. routes['A1'] = [356, 641, 190]
    '''
    routes = {}
    
    def _add_stop(dic, key, val):
        try:
            dic[key]
        except KeyError:
            dic[key] = []
        finally:
            dic[key].append(val)
    
    for stop_id in data.ID_en:
        stops_list = data.loc[data.ID_en == stop_id, ['RouteNumbers_en']].values[0][0].split('; ')
        for routes_item in stops_list:
            _add_stop(routes, routes_item, stop_id)

    return routes


def sort_routes(data, routes):
    '''
    Sort routes according to the proposed algorithm
    param data: pd.DataFrame - public transport stops data
    param routes: dict - unsorted stops ids for each route
    return: dict - sorted stops ids for each route
    '''
    
    def _sqr_dist(lat_1, long_1, lat_2, long_2):
        return (lat_2 - lat_1)**2 + (np.cos(np.radians(55)) * (long_1 - long_2))**2
        
    sorted_dict = {}    
    for route_item in routes:
        stop_array = np.array(routes[route_item])
        stop_num = stop_array.shape[0]
        stop_sqr_dist_array = np.zeros([stop_num, stop_num])
        for i in range(stop_num):
            for j in range(i+1, stop_num):
                sqr_dist_val = _sqr_dist(
                    lat_1=data.loc[data.ID_en == stop_array[i], 
                                   ['Latitude_WGS84_en']].values[0][0],
                    long_1=data.loc[data.ID_en == stop_array[i], 
                                    ['Longitude_WGS84_en']].values[0][0],
                    lat_2=data.loc[data.ID_en == stop_array[j], 
                                   ['Latitude_WGS84_en']].values[0][0],
                    long_2=data.loc[data.ID_en == stop_array[j], 
                                    ['Longitude_WGS84_en']].values[0][0])
                stop_sqr_dist_array[i, j] = sqr_dist_val
                stop_sqr_dist_array[j, i] = sqr_dist_val
        stop_sum_sqr_dist_array = np.sum(stop_sqr_dist_array, axis=1)
        current_stop = np.argmax(stop_sum_sqr_dist_array)
        route_sequence = np.array([stop_array[current_stop]])
        for i in range(stop_num):
            stop_sqr_dist_array[i, i] = np.inf
        for i in range(stop_num-1):
            next_stop = np.argmin(stop_sqr_dist_array[current_stop])
            stop_sqr_dist_array[next_stop, current_stop] = np.inf
            stop_sqr_dist_array[current_stop, next_stop] = np.inf
            current_stop = next_stop
            route_sequence = np.append(route_sequence, stop_array[next_stop])
        sorted_dict[route_item] = route_sequence  
    return sorted_dict


def get_adjacency_matrix(data, sorted_routes):
    '''
    Compute adjacency matrix for sorted routes
    param data: pd.DataFrame - public transport stops data
    param sorted_routes: dict - sorted stops ids for each route
    return: (n_samples, n_samples) - graph adjacency matrix
    '''
    stop_num = data.shape[0]
    stop_array = data.ID_en.values
    matrix = np.zeros([stop_num, stop_num])
    for route in sorted_routes:
        route_stops = sorted_routes[route]
        route_stop_num = route_stops.shape[0]
        for i_stop in range(1, route_stop_num):
            prev_stop_index = np.where(stop_array==route_stops[i_stop-1])
            next_stop_index = np.where(stop_array==route_stops[i_stop])
            matrix[prev_stop_index, next_stop_index] += 1
            matrix[next_stop_index, prev_stop_index] += 1
    return matrix

data_test = data.loc[data.ID_en.isin(['5310','4082','10651','5305','8456'])]
routes = get_routes(data_test)
sorted_routes = sort_routes(data_test, routes)
adjacency_matrix = get_adjacency_matrix(data_test, sorted_routes)
np.savetxt("hw_10_adjacency_matrix.csv", adjacency_matrix, delimiter=",")
print(sorted_routes['А125'])
data_test

In [198]:
# routes = get_routes(data)
# sorted_routes = sort_routes(data, routes)
# adjacency_matrix = get_adjacency_matrix(data, sorted_routes)

# with open('sorted_routes_hw10.pickle', 'wb') as f:
#     pickle.dump(sorted_routes, f)
    
# with open('adjacency_matrix.pickle', 'wb') as f:
#     pickle.dump(adjacency_matrix, f)

In [254]:
data_coord = data[['Latitude_WGS84_en', 'Longitude_WGS84_en']]
data_coord.head()

,Latitude_WGS84_en,Longitude_WGS84_en
0,55.782106,37.586584
1,55.790630,37.590714
2,55.714265,37.565972
3,55.712504,37.570191
4,55.713770,37.574559


map = folium.Map([55.75215, 37.61819], zoom_start=12)

coords = []
for i in sorted_routes['А125']:
    coords.append(data.loc[data.ID_en==i, ['Latitude_WGS84_en', 'Longitude_WGS84_en']].values.tolist()[0])
folium.vector_layers.PolyLine(coords).add_to(map)

print(sorted_routes['А125'])
print(coords)
data.loc[data.ID_en.isin(sorted_routes['А125'])]
map

In [260]:
def draw_clustered_map(data, labels):
    '''
    Create map with coloured clusters
    param data: pd.DataFrame - public transport stops data
    param labels: (n_samples, ) - cluster labels for each stop
    return: folium.Map - map with coloured clusters
    '''
    moscow_map = folium.Map([55.75215, 37.61819], zoom_start=12, width='100%', height='60%')
    labels = labels / np.max(labels)
    colormap = mpl.cm.get_cmap('gist_rainbow')
    
    for i in range(data.shape[0]):
        location = (data.iloc[i, 0], data.iloc[i, 1])
        color = mpl.colors.to_hex(colormap(labels[i]))
        folium.vector_layers.CircleMarker(location=location, 
                                          radius=2, color=color).add_to(moscow_map)
    
    return moscow_map

In [275]:
clustering_kmeans = KMeans(n_clusters=20)
clusters_kmean = clustering_kmeans.fit_predict(data_coord)
draw_clustered_map(data_coord, clusters_kmean)


In [276]:
clustering_spectral = SpectralClustering(n_clusters=5, affinity='precomputed')
clusters_spectral = clustering_spectral.fit_predict(adjacency_matrix)
draw_clustered_map(data_coord, clusters_spectral)


/home/constantine/anaconda3/lib/python3.8/site-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


In [272]:
eps_dbscan = (data_coord['Longitude_WGS84_en'].max() - 
                    data_coord['Longitude_WGS84_en'].min()) * 2e-2
clustering_dbscan = DBSCAN(eps=eps_dbscan)
clusters_dbscan = clustering_dbscan.fit_predict(data_coord)
draw_clustered_map(data_coord, clusters_dbscan)

In [277]:
class GraphClustering(ClusterMixin):
    def __init__(self, n_clusters=8, n_components=None, **kwargs):
        '''
        Spectral clustering algorithm
        param n_clusters: number of clusters to form
        param n_components: number of eigenvectors to use
        '''

        if n_components is None:
            n_components = n_clusters

        self.n_components = n_components
        self.kmeans = KMeans(n_clusters=n_clusters, **kwargs)

    def fit_predict(self, X, y=None):
        '''
        Perform spectral clustering from graph adjacency matrix
        and return vertex labels.
        param X: (n_samples, n_samples) - graph adjacency matrix
        return: (n_samples, ) - vertex labels
        '''

        eigenvectors = self._generate_eigenvectors(X)
        labels = self.kmeans.fit_predict(eigenvectors[:, 1:])
        return labels

    def _generate_eigenvectors(self, X):
        '''
        Compute eigenvectors for spectral clustering
        param X: (n_samples, n_samples) - graph adjacency matrix
        return: (n_samples, n_components) - eigenvectors
        '''
        diagonal_matrix = np.diag(np.sum(X, axis=1))
        laplace_matrix = diagonal_matrix - X
        eig_val, eig_vec = np.linalg.eig(laplace_matrix)
        eig_val = np.real(eig_val)
        eig_vec = np.real(eig_vec) 
        eig_vec = eig_vec[:, np.argsort(eig_val)]
        eig_val = eig_val[np.argsort(eig_val)]
        zero_eig_val_num = eig_val[np.abs(eig_val)<1e-10].shape[0]
        
        return eig_vec[:, :self.n_components]

clustering_spectral_custom = GraphClustering(n_clusters=5)
clusters_spectral_custom = clustering_spectral_custom.fit_predict(adjacency_matrix)
draw_clustered_map(data_coord, clusters_spectral_custom)